In [1]:
from pathlib import Path
import pandas as pd

# =============================================================================
# RUTAS DEL REPOSITORIO
# - Estructura esperada:
#   repo/
#     data/        (entrada, dataset anonimizado)
#     outputs/     (salidas generadas por notebooks)
#     notebooks/   (este notebook)
# =============================================================================
NOTEBOOK_DIR = Path.cwd().resolve()
REPO_ROOT = NOTEBOOK_DIR.parent

DATA_DIR = REPO_ROOT / "data"
LOCAL_INPUT = DATA_DIR / "validations_anon.csv"

# Fuente alternativa para ejecución en entornos sin el CSV local
REMOTE_INPUT = (
    "https://raw.githubusercontent.com/"
    "diferviec/transport-data-analysis/main/data/validations_anon.csv"
)

OUT_DIR = REPO_ROOT / "outputs"
OUT_DIR.mkdir(parents=True, exist_ok=True)

OUT_FILE = OUT_DIR / "validations_clean.csv"


def read_input(local_path: Path, remote_url: str) -> pd.DataFrame:
    """
    Lectura robusta del dataset:
    - Prioriza el archivo local.
    - Si no existe, intenta cargar desde el repositorio.
    """
    if local_path.exists():
        try:
            return pd.read_csv(local_path)
        except Exception as e:
            raise RuntimeError(f"Failed to read local file: {local_path} | {e}") from e

    try:
        return pd.read_csv(remote_url)
    except Exception as e:
        raise FileNotFoundError(
            "Could not read input data from either:\n"
            f"- Local:  {local_path}\n"
            f"- Remote: {remote_url}\n"
            f"Error: {e}"
        ) from e

In [2]:
# =============================================================================
# DATETIME
# Motivo:
# - En operación se observa ruido típico de exportaciones
#   y variaciones del separador.
# =============================================================================
def _clean_invisibles(s: pd.Series) -> pd.Series:
    return (
        s.astype(str).fillna("")
         .str.replace("\ufeff", "", regex=False)   # BOM
         .str.replace("\u00A0", " ", regex=False)  # NBSP
         .str.replace("\u200b", "", regex=False)   # zero-width
         .str.replace("\t", "", regex=False)
         .str.replace("\r", "", regex=False)
         .str.replace("\n", "", regex=False)
         .str.strip()
    )

def _sanitize_iso_datetime_str(s: pd.Series) -> pd.Series:
    """
    Extrae un datetime ISO desde texto “sucio”.
    Acepta:
      - YYYY-MM-DDTHH:MM:SS
      - YYYY-MM-DDTHH:MM:SS.sss...
      - YYYY-MM-DD HH:MM:SS(.sss...)  -> normaliza a 'T'
    """
    s = _clean_invisibles(s)

    # Consistencia: espacio -> 'T'
    s2 = s.str.replace(" ", "T", regex=False)

    # Extracción estricta del patrón ISO
    iso = s2.str.extract(
        r"(\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}(?:\.\d{1,6})?)",
        expand=False
    )

    # Si no coincide, se conserva el texto ya limpiado
    return iso.fillna(s2)


def parse_datetime_determinista(dt_raw: pd.Series) -> pd.Series:
    """
    Parseo determinista:
    - Limpia caracteres invisibles
    - Normaliza a ISO con 'T'
    - Parsea con formato fijo con/sin milisegundos
    """
    s = _sanitize_iso_datetime_str(dt_raw)
    has_ms = s.str.contains(r"\.", regex=True, na=False)

    dt = pd.Series(pd.NaT, index=s.index, dtype="datetime64[ns]")

    dt.loc[has_ms] = pd.to_datetime(
        s.loc[has_ms],
        format="%Y-%m-%dT%H:%M:%S.%f",
        errors="coerce",
    )
    dt.loc[~has_ms] = pd.to_datetime(
        s.loc[~has_ms],
        format="%Y-%m-%dT%H:%M:%S",
        errors="coerce",
    )

    return dt

In [3]:
# =============================================================================
# CARGA Y VALIDACIONES MÍNIMAS DE ESQUEMA
# =============================================================================
df_raw = read_input(LOCAL_INPUT, REMOTE_INPUT)

required_cols = {
    "DateTime",
    "SupportId",
    "StopPlaceShortName",
    "TransactionType",
    "ProfileName",
    "EquipmentModel",
    "ValidationStatus",
    "ValidationTicket",
}
missing = required_cols - set(df_raw.columns)
if missing:
    raise ValueError(f"Missing required columns: {sorted(missing)}")

df = df_raw.copy()

# =============================================================================
# NORMALIZACIÓN TEMPORAL
# - Mantiene únicamente registros con DateTime parseable.
# - Ordena para procesos posteriores basados en secuencia cronológica.
# =============================================================================
df["DateTime"] = parse_datetime_determinista(df["DateTime"])
df = df[df["DateTime"].notna()].copy()
df = df.sort_values("DateTime").reset_index(drop=True)

# =============================================================================
# FILTROS OPERATIVOS (REGLAS DE NEGOCIO)
# - ValidationStatus == OK: transacción aceptada a nivel operativo.
# - ValidationTicket == RETURN_CODE_OK: validación correcta del ticket/soporte.
# - Se excluye actividad de agente (operación interna / pruebas / soporte).
# =============================================================================
df = df[df["ValidationStatus"] == "OK"].copy()
df = df[df["ValidationTicket"] == "RETURN_CODE_OK"].copy()
df = df[~df["ProfileName"].isin(["Agente"])].copy()

# =============================================================================
# NORMALIZACIÓN DE ESTACIÓN
# - Homologa variaciones por codificación o captura del nombre.
# - "Bus Durán" se alinea a la estación "Durán".
# =============================================================================
df["StopPlaceShortName"] = (
    df["StopPlaceShortName"]
      .astype(str)
      .str.strip()
      .replace({
          "Bus Durán": "Durán",
          "DurÃ¡n": "Durán",
      })
)

# =============================================================================
# AGRUPACIÓN DE PERFIL
# - Simplifica el análisis: Estandar vs Preferencial.
# - Cualquier perfil distinto de Estandar se colapsa a Preferencial.
# =============================================================================
df["ProfileName"] = df["ProfileName"].where(
    df["ProfileName"] == "Estandar",
    "Preferencial"
)

# =============================================================================
# ALCANCE DEL DATASET PARA VIAJES
# - En esta etapa se excluye la "Validadora Bus" para mantener lógica basada en GATE.
# =============================================================================
df = df[df["EquipmentModel"] != "Validadora Bus"].copy()

# Columnas usadas solo como filtros (se retiran para la salida final)
df = df.drop(columns=[c for c in ["ValidationStatus", "ValidationTicket"] if c in df.columns])

# =============================================================================
# NORMALIZACIÓN DE TransactionType
# - Estandariza para lógica downstream (ENTRY/EXIT).
# - CORRESPONDENCE_ENTRY se trata como ENTRY para consistencia de eventos.
# =============================================================================
df["TransactionType"] = (
    df["TransactionType"]
      .astype(str)
      .str.strip()
      .str.upper()
      .replace({
          "CORRESPONDENCE_ENTRY": "ENTRY",
      })
)

# =============================================================================
# HIGIENE DE CAMPOS CLAVE
# - Elimina espacios y normaliza nulos típicos de lectura CSV.
# - Evita inconsistencias en joins/agrupaciones posteriores.
# =============================================================================
for c in ["SupportId", "StopPlaceShortName", "TransactionType", "ProfileName", "EquipmentModel"]:
    if c in df.columns:
        df[c] = df[c].astype(str).str.strip()
        df[c] = df[c].replace({"nan": pd.NA, "None": pd.NA, "": pd.NA})

# Orden final para análisis por tarjeta/soporte
df = df.sort_values(["SupportId", "DateTime"]).reset_index(drop=True)

# Métricas rápidas para control del resultado
print("Input rows:", f"{len(df_raw):,}")
print("Final rows:", f"{len(df):,}")
print("Min DateTime:", df["DateTime"].min())
print("Max DateTime:", df["DateTime"].max())

df.to_csv(OUT_FILE, index=False, encoding="utf-8")
print(f"Saved cleaned dataset to: {OUT_FILE.resolve()}")

Input rows: 18,564
Final rows: 14,224
Min DateTime: 2026-01-20 05:31:45.800000
Max DateTime: 2026-01-20 21:26:56.579000
Saved cleaned dataset to: C:\Users\DiegoEveraldoFernand\Python\Repositorio1\outputs\validations_clean.csv
